Ho costruito una pipeline data-driven che, a partire dallo storico del Draft NBA, pulisce il dataset, ingegnerizza le feature (ad esempio la PickBand), addestra un modello per stimare le Expected_WS di ogni giocatore e calcola il Delta (WS reale – WS attesa), permettendo di misurare in modo quantitativo steal e bust del Draft.

In [4]:
import os
import pandas as pd
import numpy as np

# ============================
# 1. Config & caricamento dati
# ============================

# Path al dataset pulito della PARTE 1
# Cambia il path se diverso nel tuo progetto
DATA_PATH = "../data/drafted_cleaned.csv"

df = pd.read_csv(DATA_PATH)

print("Shape dataset:", df.shape)
print(df.columns)

# (Opzionale) tieni solo i giocatori con almeno X minuti / partite, se hai queste colonne
# if 'MP' in df.columns:
#     df = df[df['MP'] >= 500]

# ===================================
# 2. Funzioni helper per descrizioni
# ===================================

def safe_get(row, col, default=None):
    """Restituisce row[col] se esiste e non è NaN, altrimenti default."""
    if col in row and not pd.isna(row[col]):
        return row[col]
    return default

def clean_stat(value):
    """
    Converte -1 e -100 (valori sentinella del dataset) in None.
    Lascia invariati 0, 0.0 o altri numeri validi.
    """
    if value is None or pd.isna(value):
        return None
    try:
        v = float(value)
    except:
        return None

    # Valori sentinella del tuo dataset
    if v in (-1, -100):
        return None

    return v


def normalize_percentage(v):
    try:
        v = float(v)
    except:
        return None

    # 0 rimane 0
    if v == 0:
        return 0.0

    # Frazione (0.3 → 30%)
    if 0 < v < 1:
        return v * 100

    # Caso speciale per 1 → 1%
    if v == 1:
        return 1.0

    # Già percentuale reale (es. 76, 85.7)
    return v




def format_percent(value, decimals=1):
    v = normalize_percentage(value)
    if v is None:
        return None
    return f"{v:.{decimals}f}%"


def bucketize_3p(pct):
    v = normalize_percentage(pct)
    if v is None:
        return None

    if v >= 40:
        return "ottimo tiratore da 3 punti"
    elif v >= 35:
        return "buon tiratore da 3 punti"
    elif v >= 30:
        return "tiratore da 3 punti discreto"
    else:
        return "tiratore da 3 punti poco affidabile"

def bucketize_ft(pct):
    v = normalize_percentage(pct)
    if v is None:
        return None

    if v >= 85:
        return "eccellente tiratore di tiri liberi"
    elif v >= 75:
        return "buon tiratore di tiri liberi"
    return "tiratore di liberi migliorabile"


def bucketize_reb(reb):
    """Aggettivo per rimbalzi totali."""
    if reb is None:
        return None
    try:
        v = float(reb)
    except (TypeError, ValueError):
        return None
    if v >= 10:
        return "forte rimbalzista"
    elif v >= 7:
        return "buon rimbalzista"
    elif v >= 4:
        return "rimbalzista discreto"
    else:
        return "rimbalzista non di impatto"

def bucketize_ast(ast):
    """Aggettivo per assist."""
    if ast is None:
        return None
    try:
        v = float(ast)
    except (TypeError, ValueError):
        return None
    if v >= 9:
        return "ottimo playmaker e passatore"
    elif v >= 6:
        return "buon passatore"
    else:
        return "non particolarmente orientato agli assist"

def bucketize_scorer(pts):
    """Aggettivo per capacità realizzativa (punti a partita)."""
    if pts is None:
        return None
    try:
        v = float(pts)
    except (TypeError, ValueError):
        return None
    if v >= 25:
        return "realizzatore di altissimo livello"
    elif v >= 18:
        return "buon realizzatore"
    elif v >= 12:
        return "realizzatore discreto"
    else:
        return "giocatore non focalizzato sulla realizzazione"


# =======================================
# 3. Funzione principale: riga -> testo
# =======================================

def build_player_text(row):
    pieces = []

    # Nome
    name = safe_get(row, "Player")
    if name:
        pieces.append(f"Giocatore: {name}.")

    # Provenienza
    college = safe_get(row, "College")
    if college:
        pieces.append(f"Proveniente da {college}.")

    # Draft info
    draft_year = safe_get(row, "DraftYear")
    pick = safe_get(row, "Pick")
    pickband = safe_get(row, "PickBand")
    if draft_year and pick:
        pieces.append(f"Selezionato al Draft NBA {int(draft_year)} con la pick {int(pick)} ({pickband}).")

    # Status
    status = clean_stat(safe_get(row, "Status"))
    if status == "Active":
        pieces.append("Giocatore attualmente in attività NBA.")
    elif status == "Retired":
        pieces.append("Giocatore ritirato dalla NBA.")

    # Tiro da 3
    p3 = clean_stat(safe_get(row, "3P%"))
    if p3 is not None:
        pieces.append(f"Tiro da 3: {format_percent(p3)}, {bucketize_3p(p3)}.")

    # Tiri liberi
    ft = clean_stat(safe_get(row, "FT%"))
    if ft is not None:
        pieces.append(f"Tiri liberi: {format_percent(ft)}, {bucketize_ft(ft)}.")

    # Statistiche per partita
    pts_pg = clean_stat(safe_get(row, "PTS.1"))
    if pts_pg is not None:
        pieces.append(f"Segna {pts_pg} punti a partita.")
    
    reb_pg = clean_stat(safe_get(row, "TRB.1"))
    if reb_pg is not None:
        pieces.append(f"Cattura {reb_pg} rimbalzi a partita.")

    ast_pg = clean_stat(safe_get(row, "AST.1"))
    if ast_pg is not None:
        pieces.append(f"Distribuisce {ast_pg} assist di media.")

    # Tier
    tier = safe_get(row, "Tier")
    if tier:
        pieces.append(f"Impatto complessivo classificato come: {tier}.")

    # Valori avanzati
    ws = clean_stat(safe_get(row, "WS"))
    if ws is not None:
        pieces.append(f"Win Shares totali: {ws}.")

    bpm = clean_stat(safe_get(row, "BPM"))
    if bpm and bpm != -100:
        pieces.append(f"Box Plus/Minus: {bpm}.")

    vorp = clean_stat(safe_get(row, "VORP"))
    if vorp and vorp != -100:
        pieces.append(f"VORP: {vorp}.")

    # Delta e Expected
    expected_ws = safe_get(row, "Expected_WS")
    delta = safe_get(row, "Delta")
    if expected_ws is not None:
        pieces.append(f"Valore atteso: {expected_ws:.1f} WS.")
    if delta is not None:
        pieces.append(f"Scostamento rispetto alle aspettative: {delta:.1f} WS.")

    return " ".join(pieces)



# ======================================
# 4. Applicazione al DataFrame & salvat.
# ======================================

# 1. Generazione del profilo testuale
df["text_profile"] = df.apply(build_player_text, axis=1)

# 2. Preview
for i in range(5):
    print("==== Giocatore", i, "====")
    print(df.iloc[i]["text_profile"])
    print()

# 3. Costruzione corpus
corpus_cols = ["Player", "DraftYear", "Pick", "PickBand", "text_profile", "Status"]
corpus_cols = [col for col in corpus_cols if col in df.columns]

corpus_df = df[corpus_cols].copy()

# ID univoco
corpus_df.insert(0, "player_id", corpus_df.index.astype(int))

# 4. Salvataggio
os.makedirs("../data/processed", exist_ok=True)

corpus_df.to_json("../data/processed/nba_players_corpus.json",
                  orient="records", force_ascii=False, indent=2)

corpus_df.to_csv("../data/processed/nba_players_corpus.csv", index=False)

print("Corpus salvato in:")
print(" - ../data/processed/nba_players_corpus.json")
print(" - ../data/processed/nba_players_corpus.csv")


Shape dataset: (8323, 36)
Index(['DraftYear', 'Pick', 'Tm', 'Player', 'College', 'Seasons', 'Games',
       'MP', 'PTS', 'TRB', 'AST', 'FG%', '3P%', 'FT%', 'MP.1', 'PTS.1',
       'TRB.1', 'AST.1', 'WS', 'WS/48', 'BPM', 'VORP', 'Forfeited', 'PickBand',
       'Debut', 'WS_available', 'WS/48_available', 'BPM_available',
       'VORP_available', 'MP_available', 'TRB_available', 'FG%_available',
       'FT%_available', 'MP.1_available', 'TRB.1_available', 'Status'],
      dtype='object')
==== Giocatore 0 ====
Giocatore: Clifton McNeely. Proveniente da Texas Wesleyan University. Selezionato al Draft NBA 1947 con la pick 1 (Top10). Tiri liberi: 0.0%, tiratore di liberi migliorabile. Segna 0.0 punti a partita. Cattura 0.0 rimbalzi a partita. Distribuisce 0.0 assist di media. Win Shares totali: 0.0.

==== Giocatore 1 ====
Giocatore: Glen Selbo. Proveniente da Wisconsin. Selezionato al Draft NBA 1947 con la pick 2 (Top10). Tiri liberi: 75.9%, buon tiratore di tiri liberi. Segna 3.2 punti a par

Carica il dataset pulito ottenuto nella Parte 1.

Per ogni giocatore genera una descrizione testuale in italiano che include:

informazioni anagrafiche e di provenienza (nome, college),

dettagli sul draft (anno, pick e fascia PickBand),

valutazione del tiro da 3 e dei tiri liberi tramite aggettivi qualitativi,

rimbalzi, assist e punti per partita,

metriche di impatto come Win Shares e, se presenti, Expected_WS e Delta.

Infine, salva il corpus finale nei file:

nba_players_corpus.json

nba_players_corpus.csv